In [27]:
#Ignorar os erros
import warnings
warnings.filterwarnings('ignore')
#pandas
import pandas as pd 

import matplotlib.pyplot as plt
import matplotlib as mpl
import locale
import matplotlib.ticker as ticker
from matplotlib.ticker import StrMethodFormatter
from google.cloud import storage

In [ ]:
INSTANCIANDO A CHAVE DE ACESSO PARA TER ACESSO AO STORAGE DA GOOGLE CLOUD

In [30]:
#instanciar o local que encontra a credencial
client = storage.Client.from_service_account_json('./financeiro-01-5b91f.json')

In [31]:
#Definindo função para fazer upload no storage google cloud

def upload_blob(bucket_name, source_file_name, destination_blob_name):
   #Fazer upload para o storage do google storage
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(f'Arquivo {source_file_name} enviado para {destination_blob_name} no bucket {bucket_name}.')

In [34]:
#Salvando os dados brutos na camada bronze do bucket

upload_blob('produtos_entregues','./bronze/fev_01_15.csv','bronze')

Arquivo ./bronze/fev_01_15.csv enviado para bronze no bucket produtos_entregues.


In [35]:
upload_blob('produtos_entregues','./bronze/fev_16_29.csv','bronze')

Arquivo ./bronze/fev_16_29.csv enviado para bronze no bucket produtos_entregues.


IMPORTANDO OS DATASET EM CSV, TRANSFORMAR EM DATAFRAME E CONCATENAR
PARA SER APENAS UM DATAFRAME.

# PRÉ PROCESSAMENTO DOS DADOS 

In [37]:
#importando o dataset

#Dataset 01 ao 15/02/2024
df_pacotes_entregues_1 = pd.read_csv('./bronze/fev_01_15.csv',delimiter=';')

#Dataset 16 ao 29/02/2024
df_pacotes_entregues_2 = pd.read_csv('./bronze/fev_16_29.csv',delimiter=';')

#Concatemando os dataframe
df_pacotes_entregues = pd.concat([df_pacotes_entregues_1,df_pacotes_entregues_2])

#Exibindo o dataframe

#df_pacotes_entregues.head(2)

In [38]:
#Modificando o nome das colunas 

df_pacotes_entregues.rename(columns={'Número de pedido':'ID_PEDIDO',
                                    'Franqueado':'TRANSPORTADORA',
                                     'Responsável pela entrega':'MOTORISTA',
                                     'Valor Mercadoria':'VALOR_ENCOMENDA',
                                     'CEP destino':'CEP',
                                     'Marca de assinatura':'ENTREGUE',
                                     'Endereço completo':'ENDERECO_COMPLETO',
                                     'Cidade Destino':'CIDADE',
                                     'Distrito destinatário':'BAIRRO',
                                     'Destinatário':'CLIENTE',
                                     'Horário da entrega':'DATA_ENTREGA',
                                     'Base de entrega':'GALPAO_ENTREGA',
                                     'Peso cobrável':'PESO_PRODUTO',
                                     'CNPJ':'CPF_CNPJ',
                                     'Origem':'ORIGEM',
                                     'Estado Remetente':'ESTADO_ORIGEM',
                                     'Cidade de Origem':'CIDADE_ORIGEM'}, inplace=True)

#Excluído coluna 

df_pacotes_entregues.drop(['Tipo de Produto','ORIGEM'],axis=1,inplace=True)

#Exibindo o dataframe
#df_pacotes_entregues.head(1)

Observando a dimensão do dataframe utilizando .shape
Essa função retorna o numero de linhas e o numero de colunas
(linhas, colunas)

In [39]:
#Verificando a dimensão

df_pacotes_entregues.shape

(609837, 16)

A função dataframe.isnull() retorna se existe valores nulos,
mas o retorno é em forma boleana ( False or True). Então utilizei também
.sum(), dessa forma a saída é em forma de numero.


In [40]:
#Verificando os nulos
df_pacotes_entregues.isnull().sum()

ID_PEDIDO             0
TRANSPORTADORA        0
GALPAO_ENTREGA        0
MOTORISTA             0
ENTREGUE              0
VALOR_ENCOMENDA       0
CEP                   0
ENDERECO_COMPLETO     0
CIDADE                0
BAIRRO               40
CLIENTE               0
DATA_ENTREGA          0
CPF_CNPJ              0
PESO_PRODUTO          0
ESTADO_ORIGEM         0
CIDADE_ORIGEM         0
dtype: int64

O dataframe possui 40 valores nulos no atributos Bairro e 39 no atributos
origem. Como conheço o dataset, irei tratar os missings do atributo Bairro com o valor
"BAIRRO INEXISTENTE" e os missings do atributo origem, com o mesmo nomes que possui no 
atributo 'CIDADE ORIGEM'

Abaixo especifiquei o atribuido que desejo modificar
a função .fillna() substitui os valor nulos na string ('BAIRRO INEXISTENTE')
e dentro da função, utilizei inplace-True, para salvar a alteração.

In [41]:
#Modificando os missings em 'BAIRRO INEXISTENTE'
df_pacotes_entregues['BAIRRO'].fillna('BAIRRO INEXISTENTE',inplace=True)

Para ter uma melhor consulta e melhorar os dados,
irei modificar toda as string para que elas estejam em letras maiusculas com o .upper()
o .str(), foi utilizado antes para especificar que é uma string.

In [42]:
#modificando as string para maiusculas - PADRONIZANDO OS VALORES

df_pacotes_entregues['MOTORISTA'] = df_pacotes_entregues['MOTORISTA'].str.upper()

df_pacotes_entregues['CIDADE'] = df_pacotes_entregues['CIDADE'].str.upper()

df_pacotes_entregues['BAIRRO'] = df_pacotes_entregues['BAIRRO'].str.upper()

df_pacotes_entregues['CLIENTE'] = df_pacotes_entregues['CLIENTE'].str.upper()

df_pacotes_entregues['ENTREGUE'] = df_pacotes_entregues['ENTREGUE'].str.upper()

df_pacotes_entregues['ENDERECO_COMPLETO'] = df_pacotes_entregues['ENDERECO_COMPLETO'].str.upper()

df_pacotes_entregues['GALPAO_ENTREGA'] = df_pacotes_entregues['GALPAO_ENTREGA'].str.upper()

#df_pacotes_entregues['ORIGEM'] = df_pacotes_entregues['ORIGEM'].str.upper()

df_pacotes_entregues['ESTADO_ORIGEM'] = df_pacotes_entregues['ESTADO_ORIGEM'].str.upper()

df_pacotes_entregues['CIDADE_ORIGEM'] = df_pacotes_entregues['CIDADE_ORIGEM'].str.upper()


#Exibindo o dataframe

#df_pacotes_entregues.head(2)

In [43]:
#exibindo as informações dos tipos dos atributos
df_pacotes_entregues.info()

<class 'pandas.core.frame.DataFrame'>
Index: 609837 entries, 0 to 287439
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ID_PEDIDO          609837 non-null  int64 
 1   TRANSPORTADORA     609837 non-null  object
 2   GALPAO_ENTREGA     609837 non-null  object
 3   MOTORISTA          609837 non-null  object
 4   ENTREGUE           609837 non-null  object
 5   VALOR_ENCOMENDA    609837 non-null  object
 6   CEP                609837 non-null  int64 
 7   ENDERECO_COMPLETO  609837 non-null  object
 8   CIDADE             609837 non-null  object
 9   BAIRRO             609837 non-null  object
 10  CLIENTE            609837 non-null  object
 11  DATA_ENTREGA       609837 non-null  object
 12  CPF_CNPJ           609837 non-null  int64 
 13  PESO_PRODUTO       609837 non-null  object
 14  ESTADO_ORIGEM      609837 non-null  object
 15  CIDADE_ORIGEM      609837 non-null  object
dtypes: int64(3), object(13)
m

In [44]:
#Modificando o tipo do atributo para int

df_pacotes_entregues['CPF_CNPJ'] = df_pacotes_entregues['CPF_CNPJ'].astype(int)

#exibindo
#df_pacotes_entregues.info()

Substituir a virgula por ponto, assim consigo converter o atributo para float

In [45]:
#modifica a virgula por ponto

df_pacotes_entregues['VALOR_ENCOMENDA'] = df_pacotes_entregues['VALOR_ENCOMENDA'].str.replace(',','.')

df_pacotes_entregues['PESO_PRODUTO'] = df_pacotes_entregues['PESO_PRODUTO'].str.replace(',','.')

#Exibindo
#df_pacotes_entregues.head(2)

In [46]:
#transformando o atributo em float utilizando a função .astype()

df_pacotes_entregues['VALOR_ENCOMENDA'] = df_pacotes_entregues['VALOR_ENCOMENDA'].astype(float)

df_pacotes_entregues['PESO_PRODUTO'] = df_pacotes_entregues['PESO_PRODUTO'].astype(float)

#exibindo as info do dataframe
#df_pacotes_entregues.info()

In [47]:
#transformando no tipo datetime
df_pacotes_entregues['DATA_ENTREGA'] = pd.to_datetime(df_pacotes_entregues['DATA_ENTREGA'])

#Extrair somente a data
df_pacotes_entregues['DATA_ENTREGA'] = df_pacotes_entregues['DATA_ENTREGA'].dt.date

#exibindo o dataframe
df_pacotes_entregues['DATA_ENTREGA'].head()


0    2024-02-15
1    2024-02-15
2    2024-02-15
3    2024-02-15
4    2024-02-15
Name: DATA_ENTREGA, dtype: object

In [51]:
#Transformando novamente para datetime, porque quando extrai somente a data
# o atributo é convertido para string(object)

df_pacotes_entregues['DATA_ENTREGA'] = pd.to_datetime(df_pacotes_entregues['DATA_ENTREGA'])

df_pacotes_entregues['DATA_ENTREGA'].info()

<class 'pandas.core.series.Series'>
Index: 609837 entries, 0 to 287439
Series name: DATA_ENTREGA
Non-Null Count   Dtype         
--------------   -----         
609837 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 9.3 MB


In [23]:
#exibindo os valores nulos
df_pacotes_entregues.dtypes

ID_PEDIDO                     int64
TRANSPORTADORA               object
GALPAO_ENTREGA               object
MOTORISTA                    object
ENTREGUE                     object
VALOR_ENCOMENDA             float64
CEP                           int64
ENDERECO_COMPLETO            object
CIDADE                       object
BAIRRO                       object
CLIENTE                      object
DATA_ENTREGA         datetime64[ns]
CPF_CNPJ                      int64
PESO_PRODUTO                float64
ESTADO_ORIGEM                object
CIDADE_ORIGEM                object
dtype: object

In [52]:
#Verificando a dimensão do dataframe
df_pacotes_entregues.shape

(609837, 16)

In [53]:
#Esse comando localizando o nome do veículo dentro da linha e salva na coluna TIPO_DE_VEICULO
#o nome nome referente.

df_pacotes_entregues.loc[df_pacotes_entregues['MOTORISTA'].str.contains('CARRO'),'TIPO_DE_VEICULO'] = 'CARRO'

df_pacotes_entregues.loc[df_pacotes_entregues['MOTORISTA'].str.contains('MOTO'),'TIPO_DE_VEICULO'] = 'MOTO'

df_pacotes_entregues.loc[df_pacotes_entregues['MOTORISTA'].str.contains('BIKE'),'TIPO_DE_VEICULO'] = 'BIKE'

In [55]:
#Renomenando a coluna TIPO_DE_VEICULO para VEICULO

df_pacotes_entregues.rename(columns={'TIPO_DE_VEICULO':'VEICULO'},inplace=True)

#Exibindo os nomes da colunas 
df_pacotes_entregues.columns

Index(['ID_PEDIDO', 'TRANSPORTADORA', 'GALPAO_ENTREGA', 'MOTORISTA',
       'ENTREGUE', 'VALOR_ENCOMENDA', 'CEP', 'ENDERECO_COMPLETO', 'CIDADE',
       'BAIRRO', 'CLIENTE', 'DATA_ENTREGA', 'CPF_CNPJ', 'PESO_PRODUTO',
       'ESTADO_ORIGEM', 'CIDADE_ORIGEM', 'VEICULO'],
      dtype='object')

In [56]:
#Verificando os veiculos

df_pacotes_entregues['VEICULO'].unique()

array(['CARRO', 'MOTO', 'BIKE', nan], dtype=object)

In [57]:
df_pacotes_entregues['VEICULO'].isnull().sum()

402

In [59]:
#Verificando os valores que são nulos
df_pacotes_entregues[df_pacotes_entregues['VEICULO'].isna()]

#Substituindo os valores valores nulos por 'CARRO'
df_pacotes_entregues['VEICULO'].fillna('CARRO',inplace=True)

#Verificando a existencia de mais nulos
df_pacotes_entregues['VEICULO'].isnull().sum()

0

In [97]:
#Exportando os dataframe para csv e salvando na camado Silver

df_pacotes_entregues.to_csv('./silver/entregas_fevereiro.csv',encoding='utf-8', index=False)

#Salvando os dados tratados na camada silver do bucket da google cloud

upload_blob('produtos_entregues','./silver/entregas_fevereiro.csv','silver/entregas_fevereiro.csv')

Arquivo ./silver/entregas_fevereiro.csv enviado para silver/entregas_fevereiro.csv no bucket produtos_entregues.


# TRATANDO DADOS DOS MOTORISTA 

In [98]:
#salvando os dados brutos na bronze no storage
upload_blob('produtos_entregues','./bronze/MOTORISTA.csv','bronze/motorista.csv')

Arquivo ./bronze/MOTORISTA.csv enviado para bronze/motorista.csv no bucket produtos_entregues.


In [ ]:
#Importando o dataset

df_motorista = pd.read_csv('./bronze/MOTORISTA.csv', delimiter=';')

#Exibindo o dataframe
df_motorista.head()

In [70]:
#Verificando as colunas existente
#df_motorista.info()

In [69]:
#Exibindo somente as colunas selecionadas
df_motorista_colunas = df_motorista.loc[:,['Placa','Nome da sua estação','Nome de tipo de veículo','Nome e Sobrenome CNH','CPF']]

In [72]:
#Exbibindo o novo dataframe
#df_motorista_colunas.head()

In [73]:
#Renomeando os atributos

df_motorista_colunas.rename(columns={'Placa':'PLACA',
                                     'Nome da sua estação':'NOME_BASE',
                                     'Nome de tipo de veículo':'VEICULO',
                                     'Nome e Sobrenome CNH':'MOTORISTA',
                                     'CPF':'CPF_CNPJ'},inplace=True)

In [75]:
#Verificando a dimensão
df_motorista_colunas.shape

(7547, 5)

In [76]:
#Verificando os CPF/CNPJ duplicados

df_motorista_colunas.duplicated(subset=['CPF_CNPJ']).sum()

135

In [77]:
#excluír as linhas duplicadas 

df_motorista_coluna_01 = df_motorista_colunas.drop_duplicates(subset=['CPF_CNPJ'])

#Exibir a dimensão apos a exclusão do duplicados.

df_motorista_coluna_01.shape

(7412, 5)

In [78]:
#Verificando os valores nulos

df_motorista_coluna_01.isnull().sum()

PLACA        0
NOME_BASE    7
VEICULO      0
MOTORISTA    0
CPF_CNPJ     0
dtype: int64

In [81]:
#Verificando os valores nulos
df_motorista_coluna_01[df_motorista_coluna_01['NOME_BASE'].isna()]

#Modificando os valores nulos por 'BASE DESCONHECIDA'
df_motorista_coluna_01['NOME_BASE'].fillna('BASE DESCONHECIDA', inplace=True)

#Verificando se ainda existem missings
df_motorista_coluna_01.isnull().sum()

PLACA        0
NOME_BASE    0
VEICULO      0
MOTORISTA    0
CPF_CNPJ     0
dtype: int64

In [83]:
#Verificando o tipo de veiculo que possuim no dataframe
df_motorista_coluna_01['VEICULO'].unique()

array(['FIORINO', 'Moto', 'PASSEIO', 'Utilitário', 'HR-Furgão', 'Truck',
       'Camionete', 'Furgao', 'VUC', '3/4', 'Baú', 'VAN'], dtype=object)

In [84]:
# Adequando os nomes do Veículos

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('FIORINO','CARRO')

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('PASSEIO','CARRO')

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('Utilitário','CARRO')

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('HR-Furgão','CARRO')

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('Truck','CARRO')

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('Camionete','CARRO')

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('Furgao','CARRO')

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('Furgao','CARRO')

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('VUC','CARRO')

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('3/4','CARRO')

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('Baú','CARRO')

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('VAN','CARRO')

df_motorista_coluna_01['VEICULO'] = df_motorista_coluna_01['VEICULO'].replace('Moto','MOTO')

In [86]:
#Buscando os motoristas que são tipo bike e estão especificados no nome.
#.loc serve para mostrar somente o atributos especifico
#str.contains('BIKE') -> Busca por essa palavras em uma string dentro do atributos
#,(VEICULO)] = 'BIKE' -> Muda as linha que está na pesquisa em um nome especificado 'BIKE'


df_motorista_coluna_01.loc[df_motorista_coluna_01['MOTORISTA'].str.contains('BIKE'), 'VEICULO'] = 'BIKE'

#Exibindo os tipos de veículos
df_motorista_coluna_01['VEICULO'].unique()

array(['CARRO', 'MOTO', 'BIKE'], dtype=object)

In [99]:
#Salvando localmente na pasta silver
df_motorista_coluna_01.to_csv('./silver/motorista.csv',encoding='utf-8',index=False)

#Salvando no storage do CGP na camada SILVER
upload_blob('produtos_entregues','./silver/motorista.csv','silver/motorista.csv')

Arquivo ./silver/motorista.csv enviado para silver/motorista.csv no bucket produtos_entregues.


In [95]:
#Salvando a planilha de valores das entregas no Storage do GCP na camada silver

upload_blob('produtos_entregues','./silver/valores_entregas.csv','silver/valores_entregas.csv')

Arquivo ./silver/valores_entregas.csv enviado para silver/valores_entregas.csv no bucket produtos_entregues.
